In [5]:
import pandas as pd
import re
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Initialize lemmatizer and stopwords list
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
# Load the Excel file
df = pd.read_excel('Collection of AIIMS LIST.xlsx',sheet_name='new collection by code') # Path to the Excel file. If in the same directory as code, provide file name.
df  # data obtained From various Website

,name of organization,state,tender title,document link,publish date,Unnamed: 5
0,AIIMS BHOPAL,MADHYA PRADESH,Rate Contract for the procurement of consumabl...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,18-Sep-2024,NaN
1,AIIMS BHOPAL,MADHYA PRADESH,Purchase of proprietary items - Smart scale bo...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,31-Aug-2024,NaN
2,AIIMS BHOPAL,MADHYA PRADESH,Procurement of “QIAstst DX System (Multiplexi...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,29-Aug-2024,NaN
3,AIIMS BHOPAL,MADHYA PRADESH,Procurement of proprietary items - Fletcher Su...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,22-Aug-2024,NaN
4,AIIMS BHOPAL,MADHYA PRADESH,"Procurement of (i) Karl Storz Shaver Device, (...",https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,21-Aug-2024,NaN
...,...,...,...,...,...,...
10711,AIIMS DELHI,DELHI,Purchase of Ultra Low Height Operating Table w...,https://www.aiims.edu/images/pdf/tender/OTTABL...,19/09/2024,NaN
10712,AIIMS DELHI,DELHI,Purchase of Flexible Fiberoptic Cystoscope and...,https://www.aiims.edu/images/pdf/tender/Flexib...,23/10/2023,NaN
10713,AIIMS DELHI,DELHI,High End GPU Based Laptop,https://www.aiims.edu/images/pdf/tender/GPU.pdf,13/09/2024,NaN
10714,AIIMS DELHI,DELHI,Purchase of Surgical Gown,https://www.aiims.edu/images/pdf/tender/gownm.pdf,21/08/2024,NaN


In [6]:
import pandas as pd

# Convert dates, allowing pandas to infer each date format individually
df['publish date'] = pd.to_datetime(df['publish date'], format='mixed')

# Continue with extracting the year and counting documents per year
df['Year'] = df['publish date'].dt.year
documents_per_year = df['Year'].value_counts().sort_index()

first_date = df['publish date'].min()
last_date = df['publish date'].max()

print(f'Start: {first_date}, End: {last_date}')
print(documents_per_year)


Start: 2001-01-16 20:24:00, End: 2026-07-03 00:00:00
Year
2001       2
2013       1
2014      35
2015      71
2016     189
2017     742
2018     852
2019     838
2020     839
2021     826
2022    1691
2023    2342
2024    2286
2026       2
Name: count, dtype: int64


In [10]:
# Function to clean and preprocess the 'tender title' text
def preprocess_text(text):
    # 1. Lowercasing
    text = text.lower()
    
    # 2. Replacing symbols like -, _, &, / with a space
    text = re.sub(r'[-_&/]', ' ', text)
    
    # 3. Removing punctuation and special characters (except hyphenated words)
    text = re.sub(r'[^\w\s-]', '', text)
    
    # 4. Tokenizing
    tokens = nltk.word_tokenize(text)
    
    # 5. Removing stopwords and digits, and performing lemmatization
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words and not word.isdigit()]
    
    # 6. Joining tokens back into a string
    return ' '.join(tokens)

# Apply the preprocessing to the 'tender title' column
df['tender title'] = df['tender title'].apply(preprocess_text)

# Output the dataframe with the cleaned tender titles
df[['tender title']]

,tender title
0,rate contract procurement consumables printer ...
1,purchase proprietary item smart scale board ur...
2,procurement qiastst dx system multiplexing pla...
3,procurement proprietary item fletcher suite ge...
4,procurement karl storz shaver device ii karl s...
...,...
10711,purchase ultra low height operating table radi...
10712,purchase flexible fiberoptic cystoscope ureter...
10713,high end gpu based laptop
10714,purchase surgical gown


In [11]:
# Define a list of regex patterns you want to remove
patterns_to_remove = [
    r'\b[Tt]ender\b',  # Matches both 'Tender' and 'tender'
    r'[Ss]upply[\s,]*[Ii]nstallation[\s,]*[Ii]ntegration[\s,]*[Tt]esting[\s,&]*[Cc]ommissioning',
    r'\bSIITC\b',
    r'\b[Pp]urchase\b',
    r'[Nn]otice\s*[Ii]nviting\s*[Pp]urchase\s*&',
    r'[Rr]ate\s*[Cc]ontract',
    r'[Tt]ender\s*[Dd]ocument\s*[Ss]upply\s*[Aa]nd\s*[Ii]nstallation',
    r'[Nn]otice\s*[Ii]nviting\s*[Tt]ender\s*[Ff]or\s*[Ss]upply\s*[Aa]nd\s*[Ii]nstallation\s*[Oo]f',
    r'[Tt]ender\s*[Dd]ocument\s*[Ff]or',
    r'\w+\s+[Dd]epartment\s+at\s+AIIMS\s+\w+',
    r'[Tt]o\s*AIIMS\s*\w+',
    r'[Rr]ate\s*[Cc]ontract,\s*[Ss]upply\s*&\s*[Ii]nstallation\s*[Ff]or',
    r'[Rr]ate\s*[Cc]ontract\s*[Ff]or\s*[Ss]upply\s*and\s*[Ii]nstallation\s*of',
    r'[Aa][Tt]\s*AIIMS,\s*\w+',
    r'[Ss]hort\s*[Tt]erm\s*[Tt]ender\s*[Ee]nquiry\s*[Ii]n\s*[Tt]wo\s*[Bb]id\s*[Ss]ystem\s*[Ff]or\s*[Ss]upply\s*[Oo]f',
    r'\b[Aa][Tt]\s*AIIMS\b',
    r'\b[Aa]t All India Institute of Medical Sciences\s*\w+',
    r'[Ss]upply\s*,?\s*[Ii]nstallation\s*,?\s*[Tt]esting\s*&\s*[Cc]ommissioning\s*[Oo]f',
    r'[Ss]upply\s*,?\s*[Ii]nstallation\s*,?\s*[Tt]esting\s*[Aa]nd\s*[Cc]ommissioning\s*[Oo]f',
    r'\b[Dd]epartment of [\w\s&]+ Hospital, Saket Nagar, Bhopal-\d{3}\s?\d{3}(?: \(MP\))?',
    r'[Ff]or\s*\w+\s*[Dd]epartment',
    r'[Gg]lobal\s*[Tt]ender\s*[Ee]nquiry\s*[Ff]or\s*[Ss]upply\s*[Aa]nd\s*[Ii]nstallation\s*[Oo]f',
    r'\b[Aa]t?\s*All India Institute of Medical Sciences,\s*\w+',
    r'[Aa]nnual\s*[Rr]ate\s*[Cc]ontract\s*[Oo]f',
    r'[Ff]or\s*\w+\s*\w*\s*[Dd]epartment',
    r'document',
    r'supply',
    r'installation',
    r'aiims',
    r'raipur|bhopal|rishikesh|deogarh|kalyani',
    r'procurement'
]

# Combine the patterns into a single regex pattern separated by "|"
combined_pattern = '|'.join(patterns_to_remove)

# Remove the patterns from the 'tender title' column and replace with a space " "
df['tender title'] = df['tender title'].replace(combined_pattern, ' ', regex=True)

# Optionally, strip any leading/trailing whitespace after removing patterns
df['tender title'] = df['tender title'].str.strip()

# Reset index and export to HTML
df.reset_index(drop=True, inplace=True)
df.to_html('check.html', index=True)


In [12]:

# Ensure keywords are treated as whole words with word boundaries
keywords_dict = {
    'Consumables':             ['consumables', 'consumable','consumbale' 'Consuamable', 'batteries', 'battery', 'Chemical', 'Chemicals', 
                                'reagent',  'drug', 'Milk', 'antibiotic', 'bottle',  'Sodalime',  'Reagent Kits',   'SUPPLY OF MEDICINE',
                                'fluids', 'ethanol', 'hand Rub', 'Hand Sanitizer','dopa'],

    'Hardware/Equipment':      ['wiring', 'LED', 'mechanical', 'Electrical Fittings', 'Plumbing', 'Glassware', 'surgical instruments', 'furniture', 'saw',
                                'drill', 'TRIMMER', 'Mowers', 'Pruner', 'cutter', 'Energy Meter', 'Light', 'cable', 'Electrical', 'Electronic', 
                                'line connection', 'power point', 'Sprayer', 'vacuum', 'Instrument', 'freezer', 'Electrophoresis', 'NEPHELOMETERNEPHELOMETER',
                                'PIEZO SURGERY MODULE', 'Refilling', 'refrigerated', 'Hot air oven', 'cylinder', 'cylinders', 'Mechanised', 'ro', 'Electric',
                                'Instrument', 'CRYOSPRAY', 'Warmer', 'plate', 'lamp', 'Refrigerator', 'Fire', 'weighing scale', 'UPS', 'Calibrators', 'cooling tower', 'wires', 
                                'storboscope','Oven','Forceps','cpm','sg', 'keratometer','autostainer', 'Echosheild', 'kit', 'QFT Plus Tubes', 'Serology Kits',
                                'Instruments', 'Electrophoresis', 'NEPHELOMETERNEPHELOMETER', 'PIEZO SURGERY MODULE', 'Human Skeleton', 
                                'Leyla Retractor System for Neurosurgeries', 'weighing scale', 'Bookwalter Retractor', 'mannequin','card' ,'Manikin', 'sonicator',
                                'CRYOSPRAY', 'Warmer', 'plate', 'Post Mortem Kit', 'Patient Positioner', 'COUCH', 'Taap Swedan Yantra','piezoelectric','hand piece',
                                'percutaneous dilation tracheostomy','circuit'],

    'Medical Supplies':        ['Catheter', 'surgical items', 'Hospital Store', 'Bandage', 'Disinfectant', 'syringe', 'Orthopedic Implant', 'cannula', 
                                'Abdominal Drainage Kit', 'cataract', 'Spine', 'Phaco Practice', 'Collinear Reduction Clamp', 'Gown', 'BIOSAFETY LEVEL',
                                'Orthopaedics', 'Human Skeleton', 'Leyla Retractor System for Neurosurgeries', 'Panchkarma Material', 'Tourniquet Set', 
                                'Urine', 'Bookwalter Retractor', 'Surgical Items', 'disposable', 'Sterilization', 'Post Mortem Kit', 'Patient Positioner',
                                'Patient Positioning', 'Panchakrma', 'Glove', 'Mask', 'Drainages', 'Stretcher', 'Bed', 'Related Material', 'Intrauterine Insemination',
                                'Fletcher Suite', 'Anterior Segment', 'antibody' ,'Trugnom', 'Biomlife', 'Kaya Cervi Machine','mannikin','ambu mark','telescope','microtome','cryostat'],

    'General Supplies':        ['stationery', 'Stationery', 'cleaning', 'Cleaning', 'bags', 'bag', 'containers', 'dustbin', 'BMW Bins', 'general hospital item', 
                                'mannequins', 'Manikins', 'blankets', 'blanket', 'jars', 'Staplers', 'reusable', 'injection', 'linen', 'Cartridge', 'Trays',
                                'Gloves', 'Mask', 'pad', 'pen', 'sticker', 'envelope', 'files', 'Slipper', 'Gum Boot', 'pillow', 'Foot Step', 'tape', 'Mosquito Net','kit'],

    'Facilities and Utilities': ['Air Conditioners',  'Air Conditioner', 'Air Conditioners', 'air conditional' ,'Air Conditioning', 'Air Condiction',
                                'cooling tower', 'Water Cooler', 'Water Coolers', 'vacuum', 'maintenance', 'lift', 'water supply line',
                                'Water Treatment', 'water purification', 'fire extinguisher', 'ECRP-II', 'Guest House', 'Roller Blinds',
                                'garden', 'storage', 'tank', 'nitrogen', 'liquid', 'Signboards', 'lecture', 'hall', 'flooring', 'ambulance',
                                'Buses', 'CRECHE', 'Establishment', 'bus', 'Stadium', 'GYM', 'Disposal', 'waste', 'Shelter', 'blinds', 
                                'cubicals', 'residential', 'terrace', 'floor', 'door', 'Drainage', 'painting', 'drains', 'Toilet', 'sewer', 
                                'Ceiling',  'duct insulation', 'drain', 'earthing', 'strip', 'Waterproofing', 'drinking water', 'watering', 
                                'supply line','vehicle','wardrobe', 'Water Cooler', 'Water Coolers', 'water tanker', 'water tankers',  'chiller',
                                'sprinklers', 'Water Treatment', 'water supply line', 'water purification', 'water purifier', 'Deionized Water',],

    'Services':                 ['photography', 'Videography', 'renting out', 'Cancellation', 'Trainers', 'manpower', 'Cab', 'taxi', 'sanitation', 'parking',
                                  'ADVERTISEMENT','outreach', 'Outsourcing', 'opd', 'Labour', 'skill lab', 'laundry', 'catering', 'Housekeeping', 'work', 'workers', 
                                  'Trainer', 'Insurance', 'PG', 'hiring', 'Exam', 'maintenance', 'security services', 'Examination', 'Photocopying', 'Binding', 
                                  'Lamination'],

    'Electrical Items':         ['Electrical', 'Electronic', 'line connection', 'power point', 'fan', 'UPS', 'gas'],

    'Office Supplies':          ['manual', 'Printing', 'stationary', 'Stationery', 'files', 'envelope', 'pad', 'sticker', 'book', 'ID cards'],

    'Personal Protective Equipment': ['Gloves', 'Mask', 'hand Rub', 'Hand Sanitizer', 'Gown'],

    'Furniture':                ['chair', 'Table', 'Cupboard', 'ALMIRAH', 'Stool', 'Furnishing', 'Bed', 'COUCH', 'Rack', 'Book Shelf','cabinet'],

    'Cleaning Supplies':        ['Cleaning', 'scrub station', 'Disposal', 'waste', 'dustbin', 'BMW Bins', 'Disinfectant', 'Sanitary', 'House keeping'],

    'Construction Materials': ['brick', 'bricks', 'wooden', 'plastic', 'cardboard', 'metal', 'steel', 'aluminium panel', 'interlocking tiles', 
                               'Aluminum grill'],

    'Food Supplies':            ['kitchen', 'Shampoo', 'Milk', 'Canteen', 'mess', '5cy2sri3Mess', 'food', 'cafeteria','dinner','lunch','seating','cook'],

    'Laboratory Supplies':      ['Chemicals', 'reagents', 'Glassware', 'Electrophoresis', 'NEPHELOMETERNEPHELOMETER', 'QFT Plus Tubes', 'Serology Kits',
                                  'Reagent Kits'],
                                  
    'Pharmacy':                 ['pharmacy', 'Janaushadhi Kendra', 'drug', 'antibiotic'],

    'Dental Supplies':          ['Dental Chair', 'dental instruments'],

    'Veterinary Supplies':      ['Animal House'],

    'Miscellaneous':            ['Video', 'Adhesive', 'Pest', 'IS 2631', 'household', 'Welder', 'construction', 'film', 'bod', 'signage', 'firm', 'Centre', 
                                 'GEM', 'curtains', 'napkin', 'napikins', 'civil work', 'wall', 'height', 'quotation', 'jute', 'flower', 'pot', 'trolley', 
                                 'miscellaneous',  'Corriqendum','CORRIGENDUM', 'Bike', 'exercise', 'Playing', 'curtain', 'condemned', 'cough', 'educational', 'psychological scale',
                                 'bath', 'WaterLock', 'water to', 'fixing','vehicle', 'Dressing', 'media', 'empanelment', ' Janaushadhi Kendra', 'diaper','solution','alcohol', 
                                 'beds', 'Lenin', 'Skelton', 'Sink', 'barber','shop',  'Printed Forms/Cards/Register', 'civil', 'Repair', 'cart', 'polymer','consumablesconsumables'
                                'Repairing','comment','pipe','spare parts','washbasin','Renovation','renovation','Beautification','scrap','decoration','NOC','diesel','dg set','rolls',
                                'vessel','gem','freezing','freeze','stapler','needle','scalpel','register','container','plant','cpr','filter','dvt','calibrator','flask','	solutioniodine',
                                'spare','laser','booklet','connector tube','clip','small','sweeping','mini','knee','incubator','roof','water','virus','board','aluminium','partition',
                                'hemocue', 'cuvettes','cold', 'soda', ' lime', 'canister', 'play', 'ground', 'item','workbench','newspaper','disinfection','objection','flag','dharmasala',
                                'lighting','bipolar','accessory','ice','civil','housing','interview','house','generator','rising main','rent']
}

In [13]:
import pandas as pd
import re

# Example keyword list
keywords_link = ['GeM', 'corrigendum', 'quotation', 'GEM', 'gem','compressed','Distilledwater']

# Create a case-insensitive regex pattern that matches the keywords within the URL
pattern = '|'.join(re.escape(keyword) for keyword in keywords_link)

# Filter the DataFrame to remove rows where 'document link' contains any of the keywords
filtered_df = df[~df['document link'].str.contains(pattern, case=False, regex=True, na=False)]

# Reset index after filtering
filtered_df.reset_index(drop=True, inplace=True)

# Export to an HTML file
filtered_df.to_html('check.html')

# Optional: Print the number of rows removed
print(f"Rows removed: {len(df) - len(filtered_df)}")

# Optional: Print a few examples of removed rows
print("\nExamples of removed rows:")
removed_rows = df[df['document link'].str.contains(pattern, case=False, regex=True, na=False)]
print(removed_rows['document link'].head())

Rows removed: 1455

Examples of removed rows:
154    https://aiimsbhopal.edu.in/AIIMSFiles/procurem...
409    https://aiimsbhopal.edu.in/AIIMSFiles/procurem...
412    https://aiimsbhopal.edu.in/AIIMSFiles/procurem...
413    https://aiimsbhopal.edu.in/AIIMSFiles/procurem...
414    https://aiimsbhopal.edu.in/AIIMSFiles/procurem...
Name: document link, dtype: object


In [14]:
import re

# Create a pattern by joining all the keywords from the dictionary with word boundaries
pattern = '|'.join([r'\b' + re.escape(keyword) + r'\b' for keywords in keywords_dict.values() for keyword in keywords])

# Filter the DataFrame to remove rows containing any of the exact keywords
filtered_df = filtered_df[~filtered_df['tender title'].str.contains(pattern, case=False, na=False)]

# Reset index after filtering
filtered_df.reset_index(drop=True, inplace=True)

# Display the filtered DataFrame
filtered_df.to_html('check.html')


In [15]:
# filtered_df.to_excel('../Step-2 Doc -collection with Filtered Links/2.1 Data collection/filtered_aiims_list.xlsx')

In [16]:
filtered_df

,name of organization,state,tender title,document link,publish date,Unnamed: 5
0,AIIMS BHOPAL,MADHYA PRADESH,qiastst dx system multiplexing platform syndro...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,29-Aug-2024,NaN
1,AIIMS BHOPAL,MADHYA PRADESH,jaypee digital medical e database central libr...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,18-Jul-2024,NaN
2,AIIMS BHOPAL,MADHYA PRADESH,proposal neuroblastoma cell line sh sysy pro...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,03-May-2024,NaN
3,AIIMS BHOPAL,MADHYA PRADESH,fish microscope software project funded dhr ic...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,06-Apr-2024,NaN
4,AIIMS BHOPAL,MADHYA PRADESH,autonomous human lung simulator adult pediatri...,https://aiimsbhopal.edu.in/AIIMSFiles/procurem...,05-Apr-2024,NaN
...,...,...,...,...,...,...
2527,AIIMS DELHI,DELHI,integrated difficult airway scope system adult...,https://www.aiims.edu/images/pdf/tender/Integr...,19/09/2024,NaN
2528,AIIMS DELHI,DELHI,radio frequency ablation system 01no departmen...,https://www.aiims.edu/images/pdf/tender/Ablati...,19/09/2024,NaN
2529,AIIMS DELHI,DELHI,teg thrombelastograph analyzer machine departm...,https://www.aiims.edu/images/pdf/tender/TEG Th...,21/09/2024,NaN
2530,AIIMS DELHI,DELHI,continuous renal replacement therapy crrt depa...,https://www.aiims.edu/images/pdf/tender/CRRT.pdf,21/09/2024,NaN


In [18]:
import pandas as pd

# Assuming df is your DataFrame
# Convert the 'publish date' column to datetime format
df['publish date'] = pd.to_datetime(df['publish date'], errors='coerce')

# Extract the year from the 'publish date' column
df['year'] = df['publish date'].dt.year

# Get the unique years
years_present = df['year'].dropna().unique()

# Display the unique years
print(f"Years present in the DataFrame: {sorted(years_present)}")


Years present in the DataFrame: [2014.0, 2015.0, 2016.0, 2017.0, 2018.0, 2019.0, 2020.0, 2021.0, 2022.0, 2023.0, 2024.0]


In [17]:
# import pandas as pd
# import requests

# def is_responsive(url):
#     try:
#         # Send a HEAD request to check if the link is accessible
#         response = requests.head(url, timeout=5)  # Timeout after 5 seconds
#         # Consider a link responsive if status code is between 200-399 (Success or Redirection)
#         return 200 <= response.status_code < 400
#     except requests.RequestException:
#         # Any kind of exception (timeout, connection error, etc.) means the link is not accessible
#         return False

# # Filter the DataFrame to keep only responsive URLs
# df['is_responsive'] = df['document link'].apply(is_responsive)  # Create a new column with responsiveness status
# filtered_df = df[df['is_responsive']]  # Keep only the rows where the link is responsive

# # Optionally, drop the 'is_responsive' column
# filtered_df.drop(columns=['is_responsive'], inplace=True)

# # Display the filtered DataFrame
# print(filtered_df)
